# ChatVertexAI

Note: This is separate from the Google PaLM integration. Google has chosen to offer an enterprise version of PaLM through GCP, and this supports the models made available through there. 

ChatVertexAI exposes all foundational models available in Google Cloud:

- Gemini (`gemini-pro` and `gemini-pro-vision`)
- PaLM 2 for Text (`text-bison`)
- Codey for Code Generation (`codechat-bison`)
For a full and updated list of available models visit VertexAI documentation

By default, Google Cloud [does not use](https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance#foundation_model_development) customer data to train its foundation models as part of Google Cloud`s AI/ML Privacy Commitment. More details about how Google processes data can also be found in [Google's Customer Data Processing Addendum (CDPA)](https://cloud.google.com/terms/data-processing-addendum).

To use `Google Cloud Vertex AI` PaLM you must have the `langchain-google-vertexai` Python package installed and either:
- Have credentials configured for your environment (gcloud, workload identity, etc...)
- Store the path to a service account JSON file as the GOOGLE_APPLICATION_CREDENTIALS environment variable

This codebase uses the `google.auth` library which first looks for the application credentials variable mentioned above, and then looks for system-level auth.

For more information, see: 
- https://cloud.google.com/docs/authentication/application-default-credentials#GAC
- https://googleapis.dev/python/google-auth/latest/reference/google.auth.html#module-google.auth



In [ ]:
%pip install --upgrade --quiet  langchain-google-vertexai

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI

In [ ]:
system = "You are a helpful assistant who translate English to French"
human = "Translate this sentence from English to French. I love programming."
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatVertexAI()

chain = prompt | chat
chain.invoke({})

AIMessage(content=" J'aime la programmation.")

If we want to construct a simple chain that takes user specified parameters:

In [ ]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat

chain.invoke(
    {
        "input_language": "English",
        "output_language": "Japanese",
        "text": "I love programming",
    }
)

AIMessage(content=' プログラミングが大好きです')

## Code generation chat models
You can now leverage the Codey API for code chat within Vertex AI. The model available is:
- `codechat-bison`: for code assistance

In [ ]:
chat = ChatVertexAI(
    model_name="codechat-bison", max_output_tokens=1000, temperature=0.5
)

message = chat.invoke("Write a Python function generating all prime numbers")
print(message.content)

 ```python
def generate_primes(n):
  """Generate all prime numbers up to n."""

  # Create a list of all numbers from 2 to n.
  numbers = list(range(2, n + 1))

  # Iterate over the numbers in the list.
  for i in range(len(numbers)):
    # If the number is prime, add it to the list of primes.
    if is_prime(numbers[i]):
      primes.append(numbers[i])

    # Otherwise, remove all multiples of the number from the list.
    else:
      for j in range(i + 1, len(numbers)):
        if numbers[j] % numbers[i] == 0:
          numbers.remove(numbers[j])

  # Return the list of primes.
  return primes

def is_prime(n):
  """Check if n is prime."""

  # If n is 1, it is not prime.
  if n == 1:
    return False

  # Iterate over all numbers from 2 to the square root of n.
  for i in range(2, int(n ** 0.5) + 1):
    # If n is divisible by any number from 2 to its square root, it is not prime.
    if n % i == 0:
      return False

  # If n is divisible by no number from 2 to its square root, it

## Asynchronous calls

We can make asynchronous calls via the Runnables [Async Interface](/docs/expression_language/interface).

Note: System message is not yet supported by Gemini models.

In [ ]:
# for running these examples in the notebook:
import asyncio

import nest_asyncio

nest_asyncio.apply()

In [ ]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatVertexAI(
    model_name="chat-bison", max_output_tokens=1000, temperature=0.5
)
chain = prompt | chat

asyncio.run(
    chain.ainvoke(
        {
            "input_language": "English",
            "output_language": "Sanskrit",
            "text": "I love programming",
        }
    )
)

AIMessage(content=' अहं प्रोग्रामनं प्रेमामि')

## Streaming calls

We can also stream outputs via the `stream` method:

In [ ]:
import sys

prompt = ChatPromptTemplate.from_messages(
    [("human", "List out the 5 most populous countries in the world")]
)

chat = ChatVertexAI()

chain = prompt | chat

for chunk in chain.stream({}):
    sys.stdout.write(chunk.content)
    sys.stdout.flush()

 The five most populous countries in the world are:
1. China (1.4 billion)
2. India (1.3 billion)
3. United States (331 million)
4. Indonesia (273 million)
5. Pakistan (220 million)